# 在逆向 SIMPACK 的曲线线路上，绘制整车各组件 DoFs 随时间的变化

In [ ]:
%matplotlib widget

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
plt.rcParams['font.family'] = ['Noto Sans CJK JP']

# ========== 1) 轨道中心线数据 ==========
trajectory_data = np.load('trajectory_data.npz')
s_vals   = trajectory_data['s']      # 轨道里程数组
xvals    = trajectory_data['x']      # 对应的全局 X
yvals    = trajectory_data['y']      # 全局 Y
zvals    = trajectory_data['z']      # 全局 Z
psi_vals = trajectory_data['psi']    # 轨道 yaw
phi_vals = trajectory_data['phi']    # 轨道 roll(如有超高)

left_rail  = trajectory_data['left_rail']    # (N,3) 左轨
right_rail = trajectory_data['right_rail']   # (N,3) 右轨

def get_track_pose(s):
    """
    输入轨道里程 s, 在数组 s_vals 中找到最近点索引 idx，
    返回该处轨道在全局系下的 (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T)。
    其中 pitch_T 暂时写为 0.0(无坡度)，roll_T=phi_vals[idx]。
    """
    idx = np.argmin(np.abs(s_vals - s))
    X_T = xvals[idx]
    Y_T = yvals[idx]
    Z_T = zvals[idx]
    yaw_T   = psi_vals[idx]
    pitch_T = 0.0       # 如果无坡度，可直接 0
    roll_T  = phi_vals[idx]
    return (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T)

# ========== 2) 欧拉角与坐标变换 ==========

def rot_z(yaw):
    c, s = np.cos(yaw), np.sin(yaw)
    return np.array([
        [ c, -s, 0],
        [ s,  c, 0],
        [ 0,  0, 1]
    ], dtype=float)

def rot_y(pitch):
    c, s = np.cos(pitch), np.sin(pitch)
    return np.array([
        [ c, 0,  s],
        [ 0, 1,  0],
        [-s, 0,  c]
    ], dtype=float)

def rot_x(roll):
    c, s = np.cos(roll), np.sin(roll)
    return np.array([
        [1,  0,  0],
        [0,  c, -s],
        [0,  s,  c]
    ], dtype=float)

def euler_zyx_to_matrix(yaw, pitch, roll):
    """
    矩阵 = Rz(yaw) * Ry(pitch) * Rx(roll).
    """
    return rot_z(yaw) @ rot_y(pitch) @ rot_x(roll)

def make_transform(yaw, pitch, roll, px, py, pz):
    """
    生成 4x4 的齐次变换矩阵:
      R(zyx) + 平移(px, py, pz).
    """
    T = np.eye(4)
    R = euler_zyx_to_matrix(yaw, pitch, roll)
    T[:3,:3] = R
    T[:3, 3] = [px, py, pz]
    return T

def transform_points_3d(T, xyz):
    """
    对点云 xyz(N,3) 用 4x4 齐次矩阵 T 变换，返回(N,3)。
    """
    homo = np.hstack([xyz, np.ones((len(xyz),1))])  # (N,4)
    trans = (T @ homo.T).T                          # (N,4)
    return trans[:,:3]


# ========== 3) 几何建模： 轮对 & 车体(长方体) ==========

def create_cylinder_mesh(length, radius, ntheta=30, nL=10):
    """
    沿Y轴的圆柱网格: y in [-length/2.. +length/2], 半径=radius
    返回 X, Y, Z (同 shape)，可直接 plot_surface(X,Y,Z).
    """
    y_ = np.linspace(-length/2, length/2, nL)
    th = np.linspace(0, 2*np.pi, ntheta)
    TH, Y = np.meshgrid(th, y_)
    X = radius * np.cos(TH)
    Z = radius * np.sin(TH)
    return X, Y, Z

# 车轴 & 车轮
axle_length   = 2.0
axle_radius   = 0.065
wheel_radius  = 0.43
wheel_thickness = 0.04
wheel_offset  = 0.7175  # 车轮中心相对车轴中心的 y 偏移

X_axle, Y_axle, Z_axle = create_cylinder_mesh(axle_length, axle_radius)
pts_axle = np.column_stack([X_axle.ravel(), Y_axle.ravel(), Z_axle.ravel()])

X_wh, Y_wh, Z_wh = create_cylinder_mesh(wheel_thickness, wheel_radius)
pts_wheel = np.column_stack([X_wh.ravel(), Y_wh.ravel(), Z_wh.ravel()])


def create_box_mesh(L, W, H, nx=2, ny=2, nz=2):
    """
    生成一个长方体网格(6个面),
    x in [-L/2, +L/2], y in [-W/2, +W/2], z in [0, H].
    返回 box_faces = [ (Xf, Yf, Zf), ... ]共6个面。
    """
    x_ = np.linspace(-L/2, L/2, nx)
    y_ = np.linspace(-W/2, W/2, ny)
    z_ = np.linspace(0, H, nz)  # 这里让 z 从 0 到 H

    # 上面 (z=H)
    X_top,  Y_top = np.meshgrid(x_, y_)
    Z_top = np.ones_like(X_top)*H
    # 底面 (z=0)
    X_bot,  Y_bot = np.meshgrid(x_, y_)
    Z_bot = np.zeros_like(X_bot)

    # 前面 (x=+L/2)
    Yf, Zf = np.meshgrid(y_, z_)
    Xf = np.ones_like(Yf)*(+L/2)
    # 后面 (x=-L/2)
    Yb, Zb = np.meshgrid(y_, z_)
    Xb = np.ones_like(Yb)*(-L/2)

    # 右面 (y=+W/2)
    Xr, Zr = np.meshgrid(x_, z_)
    Yr = np.ones_like(Xr)*(+W/2)
    # 左面 (y=-W/2)
    Xl, Zl = np.meshgrid(x_, z_)
    Yl = np.ones_like(Xl)*(-W/2)

    return [
        (X_top,  Y_top,  Z_top),
        (X_bot,  Y_bot,  Z_bot),
        (Xf,     Yf,     Zf),
        (Xb,     Yb,     Zb),
        (Xr,     Yr,     Zr),
        (Xl,     Yl,     Zl),
    ]

# 车体 (大盒子)
car_body_faces  = create_box_mesh(L=25.0, W=3.0, H=3.0, nx=2, ny=2, nz=2)
# 转向架 (更小的盒子)
bogie_box_faces = create_box_mesh(L=3.0,  W=2.5, H=0.5, nx=2, ny=2, nz=2)


# ========== 4) 绘制函数 ==========

def draw_wheelset(ax, T_W2G,
                  pts_axle, X_axle_shape,
                  pts_wheel, X_wheel_shape,
                  wheel_offset,
                  color_axle='magenta',
                  color_wheel='cyan',
                  alpha=1):
    """
    绘制 1 根车轴 + 2 个车轮。
    T_W2G: (轮对局部W->全局G) 的齐次变换。
    """
    # (A) 车轴
    ptsA_G = transform_points_3d(T_W2G, pts_axle)
    XA = ptsA_G[:,0].reshape(X_axle_shape)
    YA = ptsA_G[:,1].reshape(X_axle_shape)
    ZA = ptsA_G[:,2].reshape(X_axle_shape)
    ax.plot_surface(XA, YA, ZA, color=color_axle, alpha=alpha, edgecolor='none')

    # (B) 左轮
    pts_left = pts_wheel.copy()
    pts_left[:,1] += wheel_offset  # 在W系下，整体平移 y=+wheel_offset
    pts_left_G = transform_points_3d(T_W2G, pts_left)
    XL = pts_left_G[:,0].reshape(X_wheel_shape)
    YL = pts_left_G[:,1].reshape(X_wheel_shape)
    ZL = pts_left_G[:,2].reshape(X_wheel_shape)
    ax.plot_surface(XL, YL, ZL, color=color_wheel, alpha=alpha, edgecolor='none')

    # (C) 右轮
    pts_right = pts_wheel.copy()
    pts_right[:,1] -= wheel_offset
    pts_right_G = transform_points_3d(T_W2G, pts_right)
    XR = pts_right_G[:,0].reshape(X_wheel_shape)
    YR = pts_right_G[:,1].reshape(X_wheel_shape)
    ZR = pts_right_G[:,2].reshape(X_wheel_shape)
    ax.plot_surface(XR, YR, ZR, color=color_wheel, alpha=alpha, edgecolor='none')

def draw_box(ax, T_B2G, box_faces, color='orange', alpha=0.6):
    """
    绘制一个长方体(车体、转向架)。
    box_faces: create_box_mesh(...) 返回的6个面。
    """
    for (Xf, Yf, Zf) in box_faces:
        # 当前面在局部坐标下的网格点
        pts_face_local = np.column_stack([Xf.ravel(), Yf.ravel(), Zf.ravel()])
        # 变换到全局
        pts_face_global = transform_points_3d(T_B2G, pts_face_local)

        # reshape 回到 (Xf.shape)
        Xg = pts_face_global[:,0].reshape(Xf.shape)
        Yg = pts_face_global[:,1].reshape(Xf.shape)
        Zg = pts_face_global[:,2].reshape(Xf.shape)

        ax.plot_surface(Xg, Yg, Zg, color=color, alpha=alpha, edgecolor='black')


# ========== 5) 读取 SIMPACK 输出(保留各对象的 x 列) ==========

def read_simpack_6dof(file_path):
    """
    读 TSV 文件，不重命名 'y_ws01_x' -> 's_val'，
    保留所有原始列名 (y_ws01_x, y_ws02_x, y_f01_x, y_cb_x, ...)
    """
    chunks = pd.read_csv(file_path, sep='\t', float_precision='high', chunksize=10000)
    df = pd.DataFrame()
    for c in chunks:
        df = pd.concat([df, c], ignore_index=True)
    return df

file_path = os.path.join('..', 'PostAnalysis', 'Result_Y_RosRt.log')
df_wheel = read_simpack_6dof(file_path)


# ========== 6) 主绘图逻辑 ==========

fig = plt.figure(figsize=(14,6))
ax = fig.add_subplot(111, projection='3d')

# (6A) 先画轨道线
ax.plot(xvals, yvals, zvals, color='black', alpha=0.4, label='Track Center')
ax.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.4, label='Left Rail')
ax.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.4, label='Right Rail')

# 选取某个时刻行，比如最后一行
row_idx = 555
row = df_wheel.iloc[row_idx]

# (6B) 绘制4个轮对
for ws_id in [1,2,3,4]:
    # 该轮对自己在轨道方向上的位置
    s_  = row[f'y_ws{ws_id:02d}_x']
    # 轨道截面内偏移
    y_  = row[f'y_ws{ws_id:02d}_y']
    z_  = row[f'y_ws{ws_id:02d}_z']
    r_  = row[f'y_ws{ws_id:02d}_roll']
    p_  = row[f'y_ws{ws_id:02d}_pitch']
    yw_ = row[f'y_ws{ws_id:02d}_yaw']

    # 轨道->全局
    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    # 轮对->轨道
    T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)

    # 合成 轮对->全局
    T_W2G = T_T2G @ T_W2T

    # 绘制车轴+车轮
    draw_wheelset(ax, T_W2G,
                  pts_axle, X_axle.shape,
                  pts_wheel, X_wh.shape,
                  wheel_offset,
                  color_axle='magenta',
                  color_wheel='cyan',
                  alpha=1)

# (6C) 绘制2个转向架
for bg_id in [1,2]:
    s_bg  = row[f'y_f0{bg_id}_x']
    y_bg  = row[f'y_f0{bg_id}_y']
    z_bg  = row[f'y_f0{bg_id}_z']
    r_bg  = row[f'y_f0{bg_id}_roll']
    p_bg  = row[f'y_f0{bg_id}_pitch']
    yw_bg = row[f'y_f0{bg_id}_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_bg)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
    T_bg2G = T_T2G @ T_bg2T

    T_flipBogie = make_transform(0, 0, math.pi, 0, 0, 0)  # roll=180°
    # "先做本身变换 T_cb2T，再翻转 180°
    T_bg2G = T_bg2G @ T_flipBogie

    # 用较小的box模拟转向架
    draw_box(ax, T_bg2G, bogie_box_faces, color='green', alpha=0.1)

# (6D) 绘制车体
s_cb   = row['y_cb_x']
y_cb   = row['y_cb_y']
z_cb   = row['y_cb_z']
r_cb   = row['y_cb_roll']
p_cb   = row['y_cb_pitch']
yw_cb  = row['y_cb_yaw']

X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_cb)
T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)

# 合成 车体->全局
T_cb2G = T_T2G @ T_cb2T

# =============== [可选] 如果发现车体需要翻转180° ===============
#   例如 SIMPACK 里 z 是向下，模型导入后却是向上。
#   可以在 "车体局部系" 再做一个 180° 的翻转。示例(绕 X 轴翻转):
#   (可试试改成 pitch= np.pi 或 yaw= np.pi，根据实际坐标定义来调整)

T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0)  # roll=180°
# 先做车体本身变换 T_cb2T，再翻转 180°
T_cb2G = T_cb2G @ T_flipCar

# ================================================================

draw_box(ax, T_cb2G, car_body_faces, color='orange', alpha=0.05)

# ========== 7) 轴、比例、视图等设置 ==========

ax.set_xlabel("X [m]")
ax.set_ylabel("Y [m]")
ax.set_zlabel("Z [m]")
ax.set_title("Global View (Z Down), with Wheels/Bogies/Carbody")

ax.legend()

# 若想强制等比例
x_min, x_max = xvals.min(), xvals.max()
y_min, y_max = yvals.min(), yvals.max()
z_min, z_max = zvals.min(), zvals.max()
cx = 0.5*(x_min + x_max)
cy = 0.5*(y_min + y_max)
cz = 0.5*(z_min + z_max)
max_range = max(x_max - x_min, y_max - y_min, z_max - z_min)*0.5
ax.set_xlim(cx - max_range, cx + max_range)
ax.set_ylim(cy - max_range, cy + max_range)
ax.set_zlim(cz - max_range, cz + max_range)
ax.set_box_aspect((1,1,1))

# 反转 Z 轴: z 越大在图中越"低"
ax.invert_zaxis()

# 调整视角
ax.view_init(elev=25, azim=70, roll=0)

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

fig2 = plt.figure(figsize=(8,6))
ax2 = fig2.add_subplot(projection='3d')

# (1) 先画全轨道 (可选)
ax2.plot(xvals, yvals, zvals, color='black', alpha=0.8, label='Track Centerline')
ax2.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 'r-', alpha=0.8, label='Left Rail')
ax2.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 'b-', alpha=0.8, label='Right Rail')

# (2) 准备汇总所有部件的全局顶点
pts_all_in_G = []

rowF = df_wheel.iloc[row_idx]  # 最后一帧
# ============ (2A) 4 轮对 ============
for ws_id in [1,2,3,4]:
    s_  = rowF[f'y_ws{ws_id:02d}_x']
    y_  = rowF[f'y_ws{ws_id:02d}_y']
    z_  = rowF[f'y_ws{ws_id:02d}_z']
    r_  = rowF[f'y_ws{ws_id:02d}_roll']
    p_  = rowF[f'y_ws{ws_id:02d}_pitch']
    yw_ = rowF[f'y_ws{ws_id:02d}_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
    T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)
    T_W2G = T_T2G @ T_W2T

    # axle
    pts_axle_G = transform_points_3d(T_W2G, pts_axle)
    ax2.plot_surface(
        pts_axle_G[:,0].reshape(X_axle.shape),
        pts_axle_G[:,1].reshape(X_axle.shape),
        pts_axle_G[:,2].reshape(X_axle.shape),
        color='magenta', alpha=1
    )
    pts_all_in_G.append(pts_axle_G)

    # left wheel
    pts_left_local = pts_wheel.copy()
    pts_left_local[:,1] += wheel_offset
    pts_left_G = transform_points_3d(T_W2G, pts_left_local)
    ax2.plot_surface(
        pts_left_G[:,0].reshape(X_wh.shape),
        pts_left_G[:,1].reshape(X_wh.shape),
        pts_left_G[:,2].reshape(X_wh.shape),
        color='cyan', alpha=1
    )
    pts_all_in_G.append(pts_left_G)

    # right wheel
    pts_right_local = pts_wheel.copy()
    pts_right_local[:,1] -= wheel_offset
    pts_right_G = transform_points_3d(T_W2G, pts_right_local)
    ax2.plot_surface(
        pts_right_G[:,0].reshape(X_wh.shape),
        pts_right_G[:,1].reshape(X_wh.shape),
        pts_right_G[:,2].reshape(X_wh.shape),
        color='cyan', alpha=1
    )
    pts_all_in_G.append(pts_right_G)

# ============ (2B) 2 转向架 ============
for bg_id in [1,2]:
    s_bg  = rowF[f'y_f0{bg_id}_x']
    y_bg  = rowF[f'y_f0{bg_id}_y']
    z_bg  = rowF[f'y_f0{bg_id}_z']
    r_bg  = rowF[f'y_f0{bg_id}_roll']
    p_bg  = rowF[f'y_f0{bg_id}_pitch']
    yw_bg = rowF[f'y_f0{bg_id}_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_bg)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
    T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
    T_bg2G = T_T2G @ T_bg2T
    
    # 如果车体模型里需要翻转 180°(roll)，也可以这里加:
    T_flipBogie = make_transform(0, 0, math.pi, 0, 0, 0)
    T_bg2G = T_bg2G @ T_flipBogie

    # 转向架 8 个角点(或直接把整套网格变换)
    bogie_corners_local = np.array([
        [-1.5, -1.25, 0.0],
        [-1.5, -1.25, 0.5],
        [-1.5,  1.25, 0.0],
        [-1.5,  1.25, 0.5],
        [ 1.5, -1.25, 0.0],
        [ 1.5, -1.25, 0.5],
        [ 1.5,  1.25, 0.0],
        [ 1.5,  1.25, 0.5],
    ])
    bogie_corners_G = transform_points_3d(T_bg2G, bogie_corners_local)
    pts_all_in_G.append(bogie_corners_G)

    # 画转向架长方体
    draw_box(ax2, T_bg2G, bogie_box_faces, color='green', alpha=0.1)  

# ============ (2C) 车体 ============
s_cb   = rowF['y_cb_x']
y_cb   = rowF['y_cb_y']
z_cb   = rowF['y_cb_z']
r_cb   = rowF['y_cb_roll']
p_cb   = rowF['y_cb_pitch']
yw_cb  = rowF['y_cb_yaw']

X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_cb)
T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)
T_cb2G = T_T2G @ T_cb2T

# 如果需要翻转 180°
T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0)
T_cb2G = T_cb2G @ T_flipCar

car_body_corners_local = np.array([
    [-12.5, -1.5, 0.0],
    [-12.5, -1.5, 3.0],
    [-12.5,  1.5, 0.0],
    [-12.5,  1.5, 3.0],
    [ 12.5, -1.5, 0.0],
    [ 12.5, -1.5, 3.0],
    [ 12.5,  1.5, 0.0],
    [ 12.5,  1.5, 3.0],
])
car_body_corners_G = transform_points_3d(T_cb2G, car_body_corners_local)
pts_all_in_G.append(car_body_corners_G)

# 画车体
draw_box(ax2, T_cb2G, car_body_faces, color='orange', alpha=0.05)

# ============ (3) 合并并计算包围盒 ============
pts_all_in_G = np.vstack(pts_all_in_G)  # shape (N,3)
Xmin, Xmax = pts_all_in_G[:,0].min(), pts_all_in_G[:,0].max()
Ymin, Ymax = pts_all_in_G[:,1].min(), pts_all_in_G[:,1].max()
Zmin, Zmax = pts_all_in_G[:,2].min(), pts_all_in_G[:,2].max()

Xmid = 0.5*(Xmin + Xmax)
Ymid = 0.5*(Ymin + Ymax)
Zmid = 0.5*(Zmin + Zmax)
box_half_size = 0.5 * max(Xmax - Xmin, Ymax - Ymin, Zmax - Zmin)
box_half_size += 0.5  # 多留点余量

ax2.set_xlim(Xmid - box_half_size, Xmid + box_half_size)
ax2.set_ylim(Ymid - box_half_size, Ymid + box_half_size)
ax2.set_zlim(Zmid - box_half_size, Zmid + box_half_size)

ax2.set_xlabel("X [m]")
ax2.set_ylabel("Y [m]")
ax2.set_zlabel("Z [m]")
ax2.set_title("Close-up of Whole Train (Last Frame)")
ax2.invert_zaxis()  # 如果Z越大要越低
ax2.set_box_aspect((1,1,1))
ax2.view_init(elev=30, azim=-60, roll=0)
ax2.legend()

plt.tight_layout()
plt.show()

# 制作动画
- 列车在轨道上的整体运动 GlobalView_Vehicle.mp4
- 列车特写镜头 CloseupView_Vehicle.mp4
- 生成 200 帧视频约用时 3.5 min

In [ ]:
%matplotlib widget  

import numpy as np
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
plt.rcParams['font.family'] = ['Noto Sans CJK JP']  # 中文支持(可选)

# ======================= 1) 轨道中心线数据 =======================
trajectory_data = np.load('trajectory_data.npz')
s_vals   = trajectory_data['s']      # 轨道里程数组
xvals    = trajectory_data['x']      # 对应的全局 X
yvals    = trajectory_data['y']      # 全局 Y
zvals    = trajectory_data['z']      # 全局 Z
psi_vals = trajectory_data['psi']    # 轨道 yaw
phi_vals = trajectory_data['phi']    # 轨道 roll(如有超高)

left_rail  = trajectory_data['left_rail']    # (N,3) 左轨
right_rail = trajectory_data['right_rail']   # (N,3) 右轨

def get_track_pose(s):
    """
    输入轨道里程 s, 在数组 s_vals 中找到最近点索引 idx，
    返回该处轨道在全局系下的 (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T)。
    其中 pitch_T 暂时写为 0.0(无坡度)，roll_T=phi_vals[idx]。
    """
    idx = np.argmin(np.abs(s_vals - s))
    X_T = xvals[idx]
    Y_T = yvals[idx]
    Z_T = zvals[idx]
    yaw_T   = psi_vals[idx]
    pitch_T = 0.0       # 如果无坡度，可直接 0
    roll_T  = phi_vals[idx]
    return (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T)


# ======================= 2) 欧拉角与坐标变换 =======================
def rot_z(yaw):
    c, s = np.cos(yaw), np.sin(yaw)
    return np.array([
        [ c, -s, 0],
        [ s,  c, 0],
        [ 0,  0, 1]
    ], dtype=float)

def rot_y(pitch):
    c, s = np.cos(pitch), np.sin(pitch)
    return np.array([
        [ c, 0,  s],
        [ 0, 1,  0],
        [-s, 0,  c]
    ], dtype=float)

def rot_x(roll):
    c, s = np.cos(roll), np.sin(roll)
    return np.array([
        [1,  0,  0],
        [0,  c, -s],
        [0,  s,  c]
    ], dtype=float)

def euler_zyx_to_matrix(yaw, pitch, roll):
    """
    矩阵 = Rz(yaw) * Ry(pitch) * Rx(roll).
    """
    return rot_z(yaw) @ rot_y(pitch) @ rot_x(roll)

def make_transform(yaw, pitch, roll, px, py, pz):
    """
    生成 4x4 的齐次变换矩阵:
      R(zyx) + 平移(px, py, pz).
    """
    T = np.eye(4)
    R = euler_zyx_to_matrix(yaw, pitch, roll)
    T[:3,:3] = R
    T[:3, 3] = [px, py, pz]
    return T

def transform_points_3d(T, xyz):
    """
    对点云 xyz(N,3) 用 4x4 齐次矩阵 T 变换，返回(N,3)。
    """
    homo = np.hstack([xyz, np.ones((len(xyz),1))])  # (N,4)
    trans = (T @ homo.T).T                          # (N,4)
    return trans[:,:3]


# ======================= 3) 几何建模： 轮对 & 车体(长方体) =======================
def create_cylinder_mesh(length, radius, ntheta=30, nL=10):
    """
    沿Y轴的圆柱网格: y in [-length/2.. +length/2], 半径=radius
    返回 X, Y, Z (同 shape)，可直接 plot_surface(X,Y,Z).
    """
    y_ = np.linspace(-length/2, length/2, nL)
    th = np.linspace(0, 2*np.pi, ntheta)
    TH, Y = np.meshgrid(th, y_)
    X = radius * np.cos(TH)
    Z = radius * np.sin(TH)
    return X, Y, Z

def create_box_mesh(L, W, H, nx=2, ny=2, nz=2):
    """
    生成一个长方体网格(6个面),
    x in [-L/2, +L/2], y in [-W/2, +W/2], z in [0, H].
    返回 box_faces = [ (Xf, Yf, Zf), ... ]共6个面。
    """
    x_ = np.linspace(-L/2, L/2, nx)
    y_ = np.linspace(-W/2, W/2, ny)
    z_ = np.linspace(0, H, nz)  # 这里让 z 从 0 到 H

    # 上面 (z=H)
    X_top,  Y_top = np.meshgrid(x_, y_)
    Z_top = np.ones_like(X_top)*H
    # 底面 (z=0)
    X_bot,  Y_bot = np.meshgrid(x_, y_)
    Z_bot = np.zeros_like(X_bot)

    # 前面 (x=+L/2)
    Yf, Zf = np.meshgrid(y_, z_)
    Xf = np.ones_like(Yf)*(+L/2)
    # 后面 (x=-L/2)
    Yb, Zb = np.meshgrid(y_, z_)
    Xb = np.ones_like(Yb)*(-L/2)

    # 右面 (y=+W/2)
    Xr, Zr = np.meshgrid(x_, z_)
    Yr = np.ones_like(Xr)*(+W/2)
    # 左面 (y=-W/2)
    Xl, Zl = np.meshgrid(x_, z_)
    Yl = np.ones_like(Xl)*(-W/2)

    return [
        (X_top,  Y_top,  Z_top),
        (X_bot,  Y_bot,  Z_bot),
        (Xf,     Yf,     Zf),
        (Xb,     Yb,     Zb),
        (Xr,     Yr,     Zr),
        (Xl,     Yl,     Zl),
    ]


# 车轴 & 车轮
axle_length   = 2.0
axle_radius   = 0.065
wheel_radius  = 0.43
wheel_thickness = 0.04
wheel_offset  = 0.7175  # 车轮中心相对车轴中心的 y 偏移

X_axle, Y_axle, Z_axle = create_cylinder_mesh(axle_length, axle_radius)
pts_axle = np.column_stack([X_axle.ravel(), Y_axle.ravel(), Z_axle.ravel()])

X_wh, Y_wh, Z_wh = create_cylinder_mesh(wheel_thickness, wheel_radius)
pts_wheel = np.column_stack([X_wh.ravel(), Y_wh.ravel(), Z_wh.ravel()])

# 车体与转向架
car_body_faces  = create_box_mesh(L=25.0, W=3.0, H=3.0, nx=2, ny=2, nz=2)
bogie_box_faces = create_box_mesh(L=3.0,  W=2.5, H=0.5, nx=2, ny=2, nz=2)


# ======================= 4) 绘制辅助函数 =======================
def draw_wheelset(ax, T_W2G,
                  pts_axle, X_axle_shape,
                  pts_wheel, X_wheel_shape,
                  wheel_offset,
                  color_axle='magenta',
                  color_wheel='cyan',
                  alpha=1):
    """
    绘制 1 根车轴 + 2 个车轮。
    T_W2G: (轮对局部W->全局G) 的齐次变换。
    """
    # (A) 车轴
    ptsA_G = transform_points_3d(T_W2G, pts_axle)
    XA = ptsA_G[:,0].reshape(X_axle_shape)
    YA = ptsA_G[:,1].reshape(X_axle_shape)
    ZA = ptsA_G[:,2].reshape(X_axle_shape)
    ax.plot_surface(XA, YA, ZA, color=color_axle, alpha=alpha, edgecolor='none')

    # (B) 左轮
    pts_left = pts_wheel.copy()
    pts_left[:,1] += wheel_offset  # 在W系下，整体平移 y=+wheel_offset
    pts_left_G = transform_points_3d(T_W2G, pts_left)
    XL = pts_left_G[:,0].reshape(X_wheel_shape)
    YL = pts_left_G[:,1].reshape(X_wheel_shape)
    ZL = pts_left_G[:,2].reshape(X_wheel_shape)
    ax.plot_surface(XL, YL, ZL, color=color_wheel, alpha=alpha, edgecolor='none')

    # (C) 右轮
    pts_right = pts_wheel.copy()
    pts_right[:,1] -= wheel_offset
    pts_right_G = transform_points_3d(T_W2G, pts_right)
    XR = pts_right_G[:,0].reshape(X_wheel_shape)
    YR = pts_right_G[:,1].reshape(X_wheel_shape)
    ZR = pts_right_G[:,2].reshape(X_wheel_shape)
    ax.plot_surface(XR, YR, ZR, color=color_wheel, alpha=alpha, edgecolor='none')

def draw_box(ax, T_B2G, box_faces, color='orange', alpha=0.6):
    """
    绘制一个长方体(车体、转向架)。
    box_faces: create_box_mesh(...) 返回的6个面。
    """
    for (Xf, Yf, Zf) in box_faces:
        # 当前面在局部坐标下的网格点
        pts_face_local = np.column_stack([Xf.ravel(), Yf.ravel(), Zf.ravel()])
        # 变换到全局
        pts_face_global = transform_points_3d(T_B2G, pts_face_local)

        # reshape 回到 (Xf.shape)
        Xg = pts_face_global[:,0].reshape(Xf.shape)
        Yg = pts_face_global[:,1].reshape(Xf.shape)
        Zg = pts_face_global[:,2].reshape(Xf.shape)

        ax.plot_surface(Xg, Yg, Zg, color=color, alpha=alpha, edgecolor='black')


# ======================= 5) 读取 SIMPACK 输出 =======================
def read_simpack_6dof(file_path):
    """
    读 TSV 文件，不重命名 'y_ws01_x' -> 's_val'，
    保留所有原始列名 (y_ws01_x, y_ws02_x, y_f01_x, y_cb_x, ...)。
    """
    chunks = pd.read_csv(file_path, sep='\t', float_precision='high', chunksize=10000)
    df = pd.DataFrame()
    for c in chunks:
        df = pd.concat([df, c], ignore_index=True)
    return df

# ========== 假设这里已经读完了 df_wheel 的 25000 行 (size=(25000,77)) ==========
file_path = os.path.join('..', 'PostAnalysis', 'Result_Y_RosRt.log')
df_wheel = read_simpack_6dof(file_path)
# print(df_wheel.shape)  # (25000, 77)


# ======================= 6) 抽样帧索引 =======================
num_frames = 200  # 或者 2000
total_rows = len(df_wheel)  # 25000
step = total_rows // num_frames
frame_indices = range(0, total_rows, step)
# 若想更均匀可用: frame_indices = np.linspace(0, total_rows-1, num_frames, dtype=int)


# ======================= 7) 定义两个绘制函数 =======================
def plot_global_frame(ax, df_wheel, row_idx):
    """
    宏观全局视角：在给定 ax 上清空并重新绘制轨道+列车各组件。
    """
    ax.cla()

    # (1) 先画轨道线
    ax.plot(xvals, yvals, zvals, color='black', alpha=0.4, label='Track Center')
    ax.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 
            'r-', alpha=0.4, label='Left Rail')
    ax.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 
            'b-', alpha=0.4, label='Right Rail')

    # (2) 取当前帧数据
    row = df_wheel.iloc[row_idx]

    # (2A) 绘制4个轮对
    for ws_id in [1,2,3,4]:
        s_  = row[f'y_ws{ws_id:02d}_x']
        y_  = row[f'y_ws{ws_id:02d}_y']
        z_  = row[f'y_ws{ws_id:02d}_z']
        r_  = row[f'y_ws{ws_id:02d}_roll']
        p_  = row[f'y_ws{ws_id:02d}_pitch']
        yw_ = row[f'y_ws{ws_id:02d}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_)
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)
        T_W2G = T_T2G @ T_W2T

        draw_wheelset(ax, T_W2G,
                      pts_axle, X_axle.shape,
                      pts_wheel, X_wh.shape,
                      wheel_offset,
                      color_axle='magenta',
                      color_wheel='cyan',
                      alpha=1)

    # (2B) 绘制2个转向架
    for bg_id in [1,2]:
        s_bg  = row[f'y_f0{bg_id}_x']
        y_bg  = row[f'y_f0{bg_id}_y']
        z_bg  = row[f'y_f0{bg_id}_z']
        r_bg  = row[f'y_f0{bg_id}_roll']
        p_bg  = row[f'y_f0{bg_id}_pitch']
        yw_bg = row[f'y_f0{bg_id}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_bg)
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
        T_bg2G = T_T2G @ T_bg2T

        # 如果需要翻转180°（示例）
        T_flipBogie = make_transform(0, 0, math.pi, 0, 0, 0)
        T_bg2G = T_bg2G @ T_flipBogie

        draw_box(ax, T_bg2G, bogie_box_faces, color='green', alpha=0.1)

    # (2C) 绘制车体
    s_cb   = row['y_cb_x']
    y_cb   = row['y_cb_y']
    z_cb   = row['y_cb_z']
    r_cb   = row['y_cb_roll']
    p_cb   = row['y_cb_pitch']
    yw_cb  = row['y_cb_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_cb)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
    T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)
    T_cb2G = T_T2G @ T_cb2T

    T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0)
    T_cb2G = T_cb2G @ T_flipCar

    draw_box(ax, T_cb2G, car_body_faces, color='orange', alpha=0.05)

    # (3) 坐标轴设置
    ax.set_xlabel("X [m]")
    ax.set_ylabel("Y [m]")
    ax.set_zlabel("Z [m]")
    ax.set_title(f"Global View - Frame {row_idx}")

    # 让坐标范围包住轨道全长
    x_min, x_max = xvals.min(), xvals.max()
    y_min, y_max = yvals.min(), yvals.max()
    z_min, z_max = zvals.min(), zvals.max()
    cx = 0.5*(x_min + x_max)
    cy = 0.5*(y_min + y_max)
    cz = 0.5*(z_min + z_max)
    max_range = max(x_max - x_min, y_max - y_min, z_max - z_min)*0.5
    ax.set_xlim(cx - max_range, cx + max_range)
    ax.set_ylim(cy - max_range, cy + max_range)
    ax.set_zlim(cz - max_range, cz + max_range)
    ax.set_box_aspect((1,1,1))

    # 反转 Z 轴: z 越大在图中越"低"
    ax.invert_zaxis()
    ax.view_init(elev=25, azim=70, roll=0)

    # 若每帧都显示 legend，就在这加
    ax.legend()


def plot_closeup_frame(ax, df_wheel, row_idx):
    """
    列车特写视角：以较小范围包住当前车体 + 轮对 + 转向架。
    """
    ax.cla()

    # (1) 可选地先画整个轨道线(如果觉得影响观察，也可去掉)
    ax.plot(xvals, yvals, zvals, color='black', alpha=0.8, label='Track Centerline')
    ax.plot(left_rail[:,0], left_rail[:,1], left_rail[:,2], 
            'r-', alpha=0.8, label='Left Rail')
    ax.plot(right_rail[:,0], right_rail[:,1], right_rail[:,2], 
            'b-', alpha=0.8, label='Right Rail')

    # 收集该帧所有顶点 => 用于自动计算包围盒
    pts_all_in_G = []

    row = df_wheel.iloc[row_idx]

    # (2A) 4 轮对
    for ws_id in [1,2,3,4]:
        s_  = row[f'y_ws{ws_id:02d}_x']
        y_  = row[f'y_ws{ws_id:02d}_y']
        z_  = row[f'y_ws{ws_id:02d}_z']
        r_  = row[f'y_ws{ws_id:02d}_roll']
        p_  = row[f'y_ws{ws_id:02d}_pitch']
        yw_ = row[f'y_ws{ws_id:02d}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_)
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_W2T = make_transform(yw_, p_, r_, 0.0, y_, z_)
        T_W2G = T_T2G @ T_W2T

        # axle
        pts_axle_G = transform_points_3d(T_W2G, pts_axle)
        ax.plot_surface(
            pts_axle_G[:,0].reshape(X_axle.shape),
            pts_axle_G[:,1].reshape(X_axle.shape),
            pts_axle_G[:,2].reshape(X_axle.shape),
            color='magenta', alpha=1
        )
        pts_all_in_G.append(pts_axle_G)

        # left wheel
        pts_left_local = pts_wheel.copy()
        pts_left_local[:,1] += wheel_offset
        pts_left_G = transform_points_3d(T_W2G, pts_left_local)
        ax.plot_surface(
            pts_left_G[:,0].reshape(X_wh.shape),
            pts_left_G[:,1].reshape(X_wh.shape),
            pts_left_G[:,2].reshape(X_wh.shape),
            color='cyan', alpha=1
        )
        pts_all_in_G.append(pts_left_G)

        # right wheel
        pts_right_local = pts_wheel.copy()
        pts_right_local[:,1] -= wheel_offset
        pts_right_G = transform_points_3d(T_W2G, pts_right_local)
        ax.plot_surface(
            pts_right_G[:,0].reshape(X_wh.shape),
            pts_right_G[:,1].reshape(X_wh.shape),
            pts_right_G[:,2].reshape(X_wh.shape),
            color='cyan', alpha=1
        )
        pts_all_in_G.append(pts_right_G)

    # (2B) 2 转向架
    for bg_id in [1,2]:
        s_bg  = row[f'y_f0{bg_id}_x']
        y_bg  = row[f'y_f0{bg_id}_y']
        z_bg  = row[f'y_f0{bg_id}_z']
        r_bg  = row[f'y_f0{bg_id}_roll']
        p_bg  = row[f'y_f0{bg_id}_pitch']
        yw_bg = row[f'y_f0{bg_id}_yaw']

        X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_bg)
        T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
        T_bg2T = make_transform(yw_bg, p_bg, r_bg, 0.0, y_bg, z_bg)
        T_bg2G = T_T2G @ T_bg2T

        T_flipBogie = make_transform(0, 0, math.pi, 0, 0, 0)
        T_bg2G = T_bg2G @ T_flipBogie

        draw_box(ax, T_bg2G, bogie_box_faces, color='green', alpha=0.1)

        # 这里若想更精确，可把整个网格转过来append
        # 也可以只append转向架8个角点，这里演示简单append转向架盒子
        # 先示例 corners:
        bogie_corners_local = np.array([
            [-1.5, -1.25, 0.0],
            [-1.5, -1.25, 0.5],
            [-1.5,  1.25, 0.0],
            [-1.5,  1.25, 0.5],
            [ 1.5, -1.25, 0.0],
            [ 1.5, -1.25, 0.5],
            [ 1.5,  1.25, 0.0],
            [ 1.5,  1.25, 0.5],
        ])
        bogie_corners_G = transform_points_3d(T_bg2G, bogie_corners_local)
        pts_all_in_G.append(bogie_corners_G)

    # (2C) 车体
    s_cb   = row['y_cb_x']
    y_cb   = row['y_cb_y']
    z_cb   = row['y_cb_z']
    r_cb   = row['y_cb_roll']
    p_cb   = row['y_cb_pitch']
    yw_cb  = row['y_cb_yaw']

    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_cb)
    T_T2G = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)
    T_cb2T = make_transform(yw_cb, p_cb, r_cb, 0.0, y_cb, z_cb)
    T_cb2G = T_T2G @ T_cb2T

    T_flipCar = make_transform(0, 0, math.pi, 0, 0, 0)
    T_cb2G = T_cb2G @ T_flipCar

    draw_box(ax, T_cb2G, car_body_faces, color='orange', alpha=0.05)

    # 把车体8角点也加入
    car_body_corners_local = np.array([
        [-12.5, -1.5, 0.0],
        [-12.5, -1.5, 3.0],
        [-12.5,  1.5, 0.0],
        [-12.5,  1.5, 3.0],
        [ 12.5, -1.5, 0.0],
        [ 12.5, -1.5, 3.0],
        [ 12.5,  1.5, 0.0],
        [ 12.5,  1.5, 3.0],
    ])
    car_body_corners_G = transform_points_3d(T_cb2G, car_body_corners_local)
    pts_all_in_G.append(car_body_corners_G)

    # (3) 合并所有点，设置坐标轴包围范围
    pts_all_in_G = np.vstack(pts_all_in_G)
    Xmin, Xmax = pts_all_in_G[:,0].min(), pts_all_in_G[:,0].max()
    Ymin, Ymax = pts_all_in_G[:,1].min(), pts_all_in_G[:,1].max()
    Zmin, Zmax = pts_all_in_G[:,2].min(), pts_all_in_G[:,2].max()
    Xmid = 0.5*(Xmin + Xmax)
    Ymid = 0.5*(Ymin + Ymax)
    Zmid = 0.5*(Zmin + Zmax)
    box_half_size = 0.5 * max(Xmax - Xmin, Ymax - Ymin, Zmax - Zmin)
    box_half_size += 0.5  # 多留点余量

    ax.set_xlim(Xmid - box_half_size, Xmid + box_half_size)
    ax.set_ylim(Ymid - box_half_size, Ymid + box_half_size)
    ax.set_zlim(Zmid - box_half_size, Zmid + box_half_size)

    ax.set_xlabel("X [m]")
    ax.set_ylabel("Y [m]")
    ax.set_zlabel("Z [m]")
    ax.set_title(f"Close-up of Whole Train - Frame {row_idx}")
    ax.invert_zaxis()
    ax.set_box_aspect((1,1,1))
    ax.view_init(elev=30, azim=-60, roll=0)
    ax.legend()

# ======================= 8) 生成动画 (全局视角) =======================

fig_global = plt.figure(figsize=(12,6))
ax_global = fig_global.add_subplot(111, projection='3d')

def init_func_global():
    # 动画初始化用，可简单返回空
    return []

def update_func_global(frame):
    row_idx = frame_indices[frame]
    plot_global_frame(ax_global, df_wheel, row_idx)
    return []

anim_global = FuncAnimation(
    fig_global,
    update_func_global,
    frames=len(frame_indices),
    init_func=init_func_global,
    interval=100,  # 毫秒
    blit=False
)

# 保存成 mp4
anim_global.save('GlobalView_Vehicle.mp4', fps=10, dpi=100)
# 若要存 GIF => anim_global.save('global_view.gif', writer='pillow', fps=10)

# ======================= 9) 生成动画 (列车特写视角) =======================

fig_closeup = plt.figure(figsize=(8,6))
ax_closeup = fig_closeup.add_subplot(111, projection='3d')

def init_func_closeup():
    return []

def update_func_closeup(frame):
    row_idx = frame_indices[frame]
    plot_closeup_frame(ax_closeup, df_wheel, row_idx)
    return []

anim_closeup = FuncAnimation(
    fig_closeup,
    update_func_closeup,
    frames=len(frame_indices),
    init_func=init_func_closeup,
    interval=100,
    blit=False
)

anim_closeup.save('CloseupView_Vehicle.mp4', fps=10, dpi=100)

plt.show()


# 在 Python 里演示如何将轨道系 + 车辆局部 5DOF 合并成全局 6D 位姿。

## 轨道

轨道本身的三维位置和朝向存储在 trajectory_data.npz 中：

$s\_vals,\; xvals,\; yvals,\; zvals,\; \psi\_vals,\; (\phi\_vals)$

其中：
- $s\_vals$ 表示离散的里程点
- $\{xvals, yvals, zvals\}$ 是轨道在全局系的平移位置
- $\psi\_vals$、$\phi\_vals$ 是轨道在全局系的旋转角度

## 车辆部件(轮对、转向架、车体)

它们相对于"轨道系"下仅有5个自由度(2个平移 + 3个转角)，因为在轨道方向的偏移由里程 $s_{query}$ 已经管理。

具体来说：
- 局部坐标中的 $(x_{\mathrm{local}}, y_{\mathrm{local}}, z_{\mathrm{local}})$ 可简化为 $(0.0,\; y_{\mathrm{local}},\; z_{\mathrm{local}})$
- 旋转部分 $(\mathrm{roll}_{\mathrm{local}},\,\mathrm{pitch}_{\mathrm{local}},\,\mathrm{yaw}_{\mathrm{local}})$ 依旧保留3个旋转自由度

## 数据结构

在运行时，ROS 2 Subscriber可以获取**6个浮点数**：

1. $s_{\text{query}}$ — 轨道里程（1个）
2. $(y_{\mathrm{local}}, z_{\mathrm{local}}, \mathrm{roll}_{\mathrm{local}}, \mathrm{pitch}_{\mathrm{local}}, \mathrm{yaw}_{\mathrm{local}})$ — 该部件相对于轨道系的5个自由度

注意：$x_{\mathrm{local}}$ 总是0.0，因此不需要在ROS 2中额外传输。

## 坐标转换流程

1. 通过 $s_{\text{query}}$ 在轨道数据中插值，得到轨道系在全局坐标系的位置和朝向
2. 将部件的局部5自由度与轨道系位姿结合，计算部件在全局坐标系下的完整6D位姿
3. 转换过程通常使用齐次变换矩阵实现，将旋转和平移操作统一处理

In [ ]:
import numpy as np
import math

# ====================== 1) 载入轨道数据 ======================
# trajectory_data.npz 中应包含:
#   s_vals   -> 轨道里程列表(1D数组)
#   xvals    -> 全局X
#   yvals    -> 全局Y
#   zvals    -> 全局Z
#   psi_vals -> 轨道yaw (Z轴旋转)
#   phi_vals -> 轨道roll(可选,若不需要可删)
# 如有 pitch 可同理保存,也可全部设为 0
track_data = np.load('trajectory_data.npz')
s_vals   = track_data['s']
xvals    = track_data['x']
yvals    = track_data['y']
zvals    = track_data['z']
psi_vals = track_data['psi']
phi_vals = track_data['phi']  # 如果没有, 可改用0.0

# ====================== 2) 轨道查询函数 ======================
def get_track_pose(s_query: float):
    """
    在 s_vals 中找到最接近 s_query 的索引 idx,
    返回 (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T).
    这里 pitch_T 暂时当做0.0, roll_T= phi_vals[idx].
    """
    idx = np.argmin(np.abs(s_vals - s_query))

    X_T = xvals[idx]
    Y_T = yvals[idx]
    Z_T = zvals[idx]
    yaw_T   = psi_vals[idx]
    pitch_T = 0.0       # 如果轨道有 pitch, 可读 track_data['theta'][idx]
    roll_T  = phi_vals[idx]

    return (X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T)


# ====================== 3) 旋转/齐次变换工具函数 ======================
def rot_z(yaw: float):
    c, s = math.cos(yaw), math.sin(yaw)
    return np.array([
        [ c, -s,  0],
        [ s,  c,  0],
        [ 0,  0,  1]
    ], dtype=float)

def rot_y(pitch: float):
    c, s = math.cos(pitch), math.sin(pitch)
    return np.array([
        [ c,  0,  s],
        [ 0,  1,  0],
        [-s,  0,  c]
    ], dtype=float)

def rot_x(roll: float):
    c, s = math.cos(roll), math.sin(roll)
    return np.array([
        [1,  0,   0],
        [0,  c,  -s],
        [0,  s,   c]
    ], dtype=float)

def euler_zyx_to_matrix(yaw, pitch, roll):
    """
    构造 Rz(yaw)*Ry(pitch)*Rx(roll) 3x3矩阵
    """
    return rot_z(yaw) @ rot_y(pitch) @ rot_x(roll)

def make_transform(yaw, pitch, roll, px, py, pz):
    """
    生成 4x4 齐次变换:
      R(3x3) + t(3x1)
    """
    T = np.eye(4)
    R = euler_zyx_to_matrix(yaw, pitch, roll)
    T[:3,:3] = R
    T[:3, 3] = [px, py, pz]
    return T

# ====================== 4) 合并轨道与局部5DOF为全局6D ======================
def combine_pose_with_track_5d(
    s_query: float,
    y_local: float, z_local: float,
    roll_local: float, pitch_local: float, yaw_local: float
):
    """
    输入:
      - s_query: 轨道里程
      - y_local, z_local: 相对于轨道系的横移/竖移(因为 x_local=0)
      - roll_local, pitch_local, yaw_local: 相对于轨道系的绕 x/y/z 旋转
    输出:
      (X_g, Y_g, Z_g, Yaw_g, Pitch_g, Roll_g) => 全局系中的6D姿态

    内部做法:
      1) get_track_pose(s_query) 获得 轨道->全局(4x4)
      2) 构造 部件->轨道(4x4), 其中 px=0, py=y_local, pz=z_local
      3) 做矩阵相乘 => 得到部件->全局
      4) 若需要欧拉角, 从矩阵里反解
    """
    # (A) 轨道在全局系下的变换
    X_T, Y_T, Z_T, yaw_T, pitch_T, roll_T = get_track_pose(s_query)
    T_track_global = make_transform(yaw_T, pitch_T, roll_T, X_T, Y_T, Z_T)

    # (B) 部件在轨道系下: x_local=0
    T_local = make_transform(yaw_local, pitch_local, roll_local,
                             0.0, y_local, z_local)

    # (C) 总变换
    T_global = T_track_global @ T_local

    # (D) 解析 T_global 里的位移/欧拉角 (Z-Y-X)
    p_g = T_global[:3, 3]
    X_g, Y_g, Z_g = p_g[0], p_g[1], p_g[2]

    R_g = T_global[:3, :3]
    # 这里给出一个简易的"Z-Y-X"逆解:
    Yaw_g   = math.atan2(R_g[1,0], R_g[0,0])       # yaw
    # pitch 要视具体方向: pitch= asin(-R[2,0]) 也可能要正负判断
    sy = -R_g[2,0]
    Pitch_g = math.asin(sy)
    Roll_g  = math.atan2(R_g[2,1], R_g[2,2])

    return (X_g, Y_g, Z_g, Yaw_g, Pitch_g, Roll_g)

# ====================== 5) 用法举例: ROS 2 Subscriber回调 ======================

def on_subscriber_callback(msg):
    """
    假装这是 ROS 2 下的回调函数,
    我们收到了 6个浮点数据:
      s_query,
      y_local, z_local,
      roll_local, pitch_local, yaw_local
    (实际中 msg 结构由你自己定义, 这里只是示例)
    """
    s_query      = msg.s_val          # 轨道里程
    y_local      = msg.y_local
    z_local      = msg.z_local
    roll_local   = msg.roll_local
    pitch_local  = msg.pitch_local
    yaw_local    = msg.yaw_local

    # 计算全局姿态
    (X_g, Y_g, Z_g, Yaw_g, Pitch_g, Roll_g) = combine_pose_with_track_5d(
        s_query,
        y_local,
        z_local,
        roll_local,
        pitch_local,
        yaw_local
    )

    # 直接把这个 6D 数据 通过 UDP 发给 UE5
    # .... (UDP send code here) ...
    pass

# ====================== 6) 简单测试(非ROS环境) ======================

if __name__=="__main__":
    # 假装收到了: s=123.0, y=1.2, z=0.05, roll=0.01, pitch=0.02, yaw=0.2
    # 轨道方向的位移由 s 决定, 横向/竖向/转角由 local 自由度
    test_s    = 123.0
    test_y    = 1.2
    test_z    = 0.05
    test_roll = 0.01
    test_pitch= 0.02
    test_yaw  = 0.2

    Xg, Yg, Zg, Yaw_g, Pitch_g, Roll_g = combine_pose_with_track_5d(
        test_s, test_y, test_z, test_roll, test_pitch, test_yaw
    )
    print("Global Pose => ",
          f"pos=({Xg:.3f}, {Yg:.3f}, {Zg:.3f}),",
          f"eulerZyx=(yaw={Yaw_g:.2f}, pitch={Pitch_g:.2f}, roll={Roll_g:.2f})")
